# nflfastR Data Pipeline

Here's the ETL process I send every nflfastR file through to ensure it's easier to queri using SQL engines. This pipeline standardizes formats, eliminates unnecessary CASTs, etc.

In [113]:
import polars as pl
import duckdb

YEAR = 1999

df = pl.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   has_header=True,
                   infer_schema_length = 10000)

In [114]:
print(df)
print(df.schema)
print(df.dtypes)

shape: (46_136, 372)
┌─────────┬───────────────┬─────────────┬───────────┬───┬──────────────┬─────────┬───────┬─────────┐
│ play_id ┆ game_id       ┆ old_game_id ┆ home_team ┆ … ┆ xyac_success ┆ xyac_fd ┆ xpass ┆ pass_oe │
│ ---     ┆ ---           ┆ ---         ┆ ---       ┆   ┆ ---          ┆ ---     ┆ ---   ┆ ---     │
│ i64     ┆ str           ┆ i64         ┆ str       ┆   ┆ str          ┆ str     ┆ str   ┆ str     │
╞═════════╪═══════════════╪═════════════╪═══════════╪═══╪══════════════╪═════════╪═══════╪═════════╡
│ 35      ┆ 1999_01_ARI_P ┆ 1999091200  ┆ PHI       ┆ … ┆ null         ┆ null    ┆ null  ┆ null    │
│         ┆ HI            ┆             ┆           ┆   ┆              ┆         ┆       ┆         │
│ 60      ┆ 1999_01_ARI_P ┆ 1999091200  ┆ PHI       ┆ … ┆ null         ┆ null    ┆ null  ┆ null    │
│         ┆ HI            ┆             ┆           ┆   ┆              ┆         ┆       ┆         │
│ 82      ┆ 1999_01_ARI_P ┆ 1999091200  ┆ PHI       ┆ … ┆ null        

In [115]:
full = df.sort(["game_id", "play_id"]).unique(subset=["game_id", "play_id"], keep="last")

In [116]:
full.write_parquet(
    f"nflfastR_cleaned/pbp_{YEAR}_full.parquet",
    compression="zstd",
    statistics=True
)


In [117]:
import duckdb

con = duckdb.connect("pbp.duckdb")

con.execute(f"""
    CREATE OR REPLACE TABLE pbp_plays AS
    SELECT * FROM read_parquet('nflfastR_cleaned\pbp_{YEAR}_full.parquet');
""")

df_preview = con.execute("SELECT * FROM pbp_plays LIMIT 5").pl()

con.close()

print(df_preview)

<>:8: SyntaxWarning: invalid escape sequence '\p'
<>:8: SyntaxWarning: invalid escape sequence '\p'
C:\Users\RaymondCarpenter\AppData\Local\Temp\ipykernel_19768\3604039247.py:8: SyntaxWarning: invalid escape sequence '\p'
  """)


shape: (5, 372)
┌─────────┬───────────────┬─────────────┬───────────┬───┬──────────────┬─────────┬───────┬─────────┐
│ play_id ┆ game_id       ┆ old_game_id ┆ home_team ┆ … ┆ xyac_success ┆ xyac_fd ┆ xpass ┆ pass_oe │
│ ---     ┆ ---           ┆ ---         ┆ ---       ┆   ┆ ---          ┆ ---     ┆ ---   ┆ ---     │
│ i64     ┆ str           ┆ i64         ┆ str       ┆   ┆ str          ┆ str     ┆ str   ┆ str     │
╞═════════╪═══════════════╪═════════════╪═══════════╪═══╪══════════════╪═════════╪═══════╪═════════╡
│ 176     ┆ 1999_01_ARI_P ┆ 1999091200  ┆ PHI       ┆ … ┆ null         ┆ null    ┆ null  ┆ null    │
│         ┆ HI            ┆             ┆           ┆   ┆              ┆         ┆       ┆         │
│ 281     ┆ 1999_01_ARI_P ┆ 1999091200  ┆ PHI       ┆ … ┆ null         ┆ null    ┆ null  ┆ null    │
│         ┆ HI            ┆             ┆           ┆   ┆              ┆         ┆       ┆         │
│ 305     ┆ 1999_01_ARI_P ┆ 1999091200  ┆ PHI       ┆ … ┆ null         ┆ nu